# Read in CCES, ANES, NAES


In [5]:
options(warn=-1)
rm(list = ls())
setwd("/Users/Chris/Dropbox/masterData/")
# read SPSS data
library(foreign)
library(dplyr)
library(ggplot2)
library(ggrepel)
library(stringr)
library(tidyr



## Data Format

- stateFIPS: NUMERIC. This is the FIPS code for the state
- year: NUMERIC. This is the year of the survey
- pid3: CHARACTER. This is the 3-category party identification variable. Leaners are Independent.
- ideology: CHARACTER. This is the 3-category ideoloogy variable.
- race: CHARACTER. ("White", "Black", "Asian", "Native", "Other", "Mixed", "")
- hispanic: NUMERIC. The respondent identifies as hispanic.
- age: NUMERIC. Respondent age.
- gender: CHARACTER. ["Female", "Male", "Other", ""]
- org: CHARACTER. ["CCES", "ANES", "NAES"]
- idYear: NUMERIC. This is the within survey respondent ID.


# The 2000 Annenberg Data

The 2000 data come in different cross sections, 5 in total. So I load these, create a function to recode everything identically, and consisent with the above coding scheme. I then bind them together.


In [6]:
# Write a loop to open G1.sav through G5.sav
# and save them as dataframes
for (i in 1:5) {
  assign(paste0("G", i), read.spss(paste0("NAES/dat2000/G", i, ".sav"), add.undeclared.levels = "no", to.data.frame = "TRUE"))
}


In [7]:
create2000 = function(data = G1){
  df = data %>%
  as.data.frame() %>%
  mutate(stateFIPS = as.character(cst)) %>%
  # Create state fips code from two letter abbreviation
  mutate(stateFIPS = case_when(
    stateFIPS == "AL" ~ "01",
    stateFIPS == "AK" ~ "02",
    stateFIPS == "AZ" ~ "04",
    stateFIPS == "AR" ~ "05",
    stateFIPS == "CA" ~ "06",
    stateFIPS == "CO" ~ "08",
    stateFIPS == "CT" ~ "09",
    stateFIPS == "DE" ~ "10",
    stateFIPS == "FL" ~ "12",
    stateFIPS == "GA" ~ "13",
    stateFIPS == "HI" ~ "15",
    stateFIPS == "ID" ~ "16",
    stateFIPS == "IL" ~ "17",
    stateFIPS == "IN" ~ "18",
    stateFIPS == "IA" ~ "19",
    stateFIPS == "KS" ~ "20",
    stateFIPS == "KY" ~ "21",
    stateFIPS == "LA" ~ "22",
    stateFIPS == "ME" ~ "23",
    stateFIPS == "MD" ~ "24",
    stateFIPS == "MA" ~ "25",
    stateFIPS == "MI" ~ "26",
    stateFIPS == "MN" ~ "27",
    stateFIPS == "MS" ~ "28",
    stateFIPS == "MO" ~ "29",
    stateFIPS == "MT" ~ "30",
    stateFIPS == "NE" ~ "31",
    stateFIPS == "NV" ~ "32",
    stateFIPS == "NH" ~ "33",
    stateFIPS == "NJ" ~ "34",
    stateFIPS == "NM" ~ "35",
    stateFIPS == "NY" ~ "36",
    stateFIPS == "NC" ~ "37",
    stateFIPS == "ND" ~ "38",
    stateFIPS == "OH" ~ "39",
    stateFIPS == "OK" ~ "40",
    stateFIPS == "OR" ~ "41",
    stateFIPS == "PA" ~ "42",
    stateFIPS == "RI" ~ "44",
    stateFIPS == "SC" ~ "45",
    stateFIPS == "SD" ~ "46",
    stateFIPS == "TN" ~ "47",
    stateFIPS == "TX" ~ "48",
    stateFIPS == "UT" ~ "49",
    stateFIPS == "VT" ~ "50",
    stateFIPS == "VA" ~ "51",
    stateFIPS == "WA" ~ "53",
    stateFIPS == "WV" ~ "54",
    stateFIPS == "WI" ~ "55",
    stateFIPS == "WY" ~ "56",
    TRUE ~ stateFIPS
  )) %>%
  mutate(pid3 = recode(as.character(cv01), "Democrat" = "Democrat", "Republican" = "Republican", "Independent" = "Independent", "Something else" = "Independent", "No answer" = "", "Dont know" = "", "Verbatim" = "")) %>%
  mutate(ideology = recode(as.character(cv04),
    "Very conservative" = "Conservative", "Conservative" = "Conservative", "Moderate" = "Moderate", "Liberal" = "Liberal", "Very liberal" = "Liberal",
    "No answer" = "", "Dont know" = ""
  )) %>%
  mutate(year = 2000) %>%
  mutate(idYear = row_number()) %>%
  select(stateFIPS, idYear, year, pid3, ideology, race = cw03, hispanic = cw04, age = cw02, sex = cw01) %>%
  mutate(race = as.character(race)) %>%
  mutate(sex = as.character(sex)) %>%
  mutate(race = ifelse(race %in% c("No answer", "Verbatim", "Dont know"), "", race)) %>%
  mutate(sex = ifelse(sex %in% c("No answer", "Verbatim", "Dont know"), "", sex)) %>%
  mutate(hispanic = ifelse(hispanic == "Yes", 1, 0)) %>%
  mutate(org ="NAES")
  return(df) }


In [9]:
# Construct Data
dat00 <-
  rbind(
    create2000(G1),
    create2000(G2),
    create2000(G3),
    create2000(G4),
    create2000(G5)
  )


# The 2004 National Annenberg Election Survey

The formatting is a bit different for this one. I load it in, and then recode it to match the above coding scheme.

In [14]:
NAES04 <- read.spss("NAES/DataNRCS.sav", add.undeclared.levels = "no", to.data.frame = "TRUE")

dat04 <- NAES04 %>%
  as.data.frame() %>%
  mutate(stateFIPS = as.character(cST)) %>%
  mutate(stateFIPS = str_extract(stateFIPS, "\\d+")) %>%
  mutate(stateFIPS = str_pad(dat04$stateFIPS, width = 2, side = "left", pad = "0")) %>%
  mutate(pid3 = recode(as.character(cMA01), "Democrat" = "Democrat", "Republican" = "Republican", "Independent" = "Independent", "Something else" = "Independent", "Refused" = "", "Don't know" = "")) %>%
  # Recode ideology a character correctl
  mutate(ideology = recode(as.character(cMA06), "Very conservative" = "Conservative", "Conservative" = "Conservative", "Moderate" = "Moderate", "Liberal" = "Liberal", "Very liberal" = "Liberal", "Refused" = "", "Don't know" = "" )) %>%
  mutate(year = 2004) %>%
  mutate(idYear = c(1:nrow(.))) %>%
  select(stateFIPS, idYear, year, pid3, ideology, race = cWC03, hispanic = cWC01, age = cWA02, sex = cWA01) %>%
  mutate(hispanic = ifelse(hispanic == "Yes", 1, 0)) %>%
  mutate(race = as.character(race)) %>%
  mutate(race = ifelse(race %in% c("No answer", "Verbatim", "Don't know", "Refused"), "", race)) %>%
  mutate(race = ifelse(race == "American Indian", "Native", race)) %>%
  mutate(org = "NAES")
head(dat04)


,stateFIPS,idYear,year,pid3,ideology,race,hispanic,age,sex,org
,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<fct>,<chr>
1,34,1,2004,Democrat,Conservative,Other,1,70,Female,NAES
2,34,2,2004,Democrat,Moderate,White,0,54,Female,NAES
3,23,3,2004,Independent,Moderate,White,0,74,Male,NAES
4,54,4,2004,Republican,Moderate,Other,0,73,Female,NAES
5,39,5,2004,Independent,Liberal,White,0,48,Female,NAES
6,25,6,2004,Democrat,Liberal,White,0,58,Male,NAES


In [16]:
# Extract the state FIPS code from the state variable
NAES08 <- read.spss("NAES/naes08-phone-nat-rcs-data-compact.sav", add.undeclared.levels = "no", to.data.frame = "TRUE")

In [17]:

NAES08$pid3 <- recode(as.numeric(NAES08$MA01), `1` = "Democrat", `2` = "Republican", `3` = "Independent", `4` = "Independent", `5` = "", `6` = "")
NAES08$ideology <- recode(as.numeric(NAES08$MA04), `1` = "Conservative", `2` = "Conservative", `3` = "Moderate", `4` = "Liberal", `5` = "Liberal", `6` = "", `7` = "")
NAES08$Race <- recode(as.numeric(NAES08$WC03),
  `1` = "White",
  `2` = "Black",
  `3` = "Asian",
  `4` = "Native",
  `5` = "Other",
  `6` = "Mixed",
  `7` = "Other",
  `8` = "",
  `9` = ""
)
NAES08$year <- 2008
NAES08$Hispanic <- recode(as.numeric(NAES08$WC03),
  "1" = "1",
  "2" = "1",
  "3" = "0",
  "4" = "0",
  "5" = "1",
  "6" = "0",
  "7" = "0",
  "8" = "0",
  "9" = "0"
) %>% as.numeric()

NAES08$age <- NAES08$WA02
NAES08$sex <- NAES08$WA01
NAES08$year <- 2008
NAES08$org <- "NAES"


In [21]:
NAES08$stateNAME <- str_extract_all(NAES08$WFc01, "[A-Z][A-Z]") %>% unlist()
recode_state_fips <- function(state_abbr) {
  state_fips_codes <- c(
    "AL" = 1, "AZ" = 4, "AR" = 5, "CA" = 6, "CO" = 8, "CT" = 9, "DE" = 10, "DC" = 11,
    "FL" = 12, "GA" = 13, "ID" = 16, "IL" = 17, "IN" = 18, "IA" = 19, "KS" = 20,
    "KY" = 21, "LA" = 22, "ME" = 23, "MD" = 24, "MA" = 25, "MI" = 26, "MN" = 27,
    "MS" = 28, "MO" = 29, "MT" = 30, "NE" = 31, "NV" = 32, "NH" = 33, "NJ" = 34,
    "NM" = 35, "NY" = 36, "NC" = 37, "ND" = 38, "OH" = 39, "OK" = 40, "OR" = 41,
    "PA" = 42, "RI" = 44, "SC" = 45, "SD" = 46, "TN" = 47, "TX" = 48, "UT" = 49,
    "VT" = 50, "VA" = 51, "WA" = 53, "WV" = 54, "WI" = 55, "WY" = 56
  )

  # Convert state abbreviation to FIPS code
  fips_code <- state_fips_codes[state_abbr]
  return(fips_code)
}

NAES08$stateFIPS <- recode_state_fips(NAES08$stateNAME)
NAES08$stateFIPS <- str_pad(NAES08$stateFIPS, width = 2, side = "left", pad = "0")
NAES08$idYear <- c(1:nrow(NAES08))
NAES08$age <- ifelse(NAES08$age > 100, NA, NAES08$age)


In [22]:

NAES08 <- NAES08 %>% select(hispanic = Hispanic, year, race = Race, ideology, pid3, age, sex, year, org, stateFIPS, idYear)


In [29]:
library(readstata13)
CCES <- read.dta13("CCES/common/cumulative_2006-2022.dta", convert.factors = "FALSE")


CCESData <- CCES %>%
  mutate(pid3 = recode(as.character(pid3), `1` = "Democrat", `2` = "Republican", `3` = "Independent", `4` = "Independent", `5` = "")) %>%
  mutate(ideology = recode(as.character(ideo5), `1` = "Liberal", `2` = "Liberal", `3` = "Moderate", `4` = "Conservative", `5` = "Conservative", `6` = "")) %>%
  mutate(sex = recode(sex, `1` = "Male", `2` = "Female", `3` = "")) %>%
  mutate(age = age) %>%
  mutate(RACE = recode(race, `1` = "White", `2` = "Black", `3` = "Other", `4` = "Asian", `5` = "Native", `6` = "Other", `7` = "Other", `8` = "Other")) %>%
  mutate(stateFIPS = as.character(state)) %>%
  mutate(year = year) %>%
  mutate(org = "CCES") %>%
  mutate(idYear = c(1:nrow(.))) %>%
  mutate(hispanic = ifelse(as.numeric(race) == 3, 1, 0)) %>%
  select(pid3, ideology, sex, age, race = RACE, stateFIPS, year, org, idYear, hispanic) %>%
  mutate(stateFIPS = str_pad(stateFIPS, width = 2, side = "left", pad = "0"))


CCESData %>% head()


,pid3,ideology,sex,age,race,stateFIPS,year,org,idYear,hispanic
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<int>,<dbl>
1,Democrat,Liberal,Female,32,White,37,2006,CCES,1,0
2,Independent,Moderate,Male,49,White,39,2006,CCES,2,0
3,Democrat,Liberal,Female,54,White,34,2006,CCES,3,0
4,Democrat,Liberal,Female,34,Black,17,2006,CCES,4,0
5,Democrat,Liberal,Male,20,White,36,2006,CCES,5,0
6,Independent,Liberal,Female,27,White,48,2006,CCES,6,0


In [30]:
# ANES
# print the working directory
getwd()
anes <- read.csv("ANES_CrossSections/cumulative1023/anes/cumu.csv")

[1] "/Users/Chris/Dropbox/masterData"

In [33]:
# Recodes, stateFIPS, idYear, pid3, ideology, race, hisanic, age, sex, org a 10 column
ANES <- anes %>%
  mutate(pid3 = recode(as.numeric(VCF0302), `1` = "Republican", `2` = "Independent", `3` = "Independent", `4` = "Independent", `5` = "Democrat", `8` = "", `9` = "")) %>%
  mutate(ideology = recode(VCF0803,
    `1` = "Liberal",
    `2` = "Liberal", `3` = "Liberal", `4` = "Moderate", `5` = "Conservative",
    `5` = "Conservative", `6` = "Conservative", `7` = "Conservative", `9` = "", `0` = ""
  )) %>%
  mutate(age = car::recode(as.numeric(VCF0102), "1 =21; 2=30; 3=40; 4=50; 5=60; 6=70; 7=80")) %>%
  mutate(gender = car::recode(as.numeric(VCF0104), "1 = 'Male'; 2 = 'Female'")) %>%
  mutate(race = recode(VCF0105a, `1` = "White", `2` = "Black", `3` = "Asian", `4` = "Native", `5` = "Other", `6` = "Other", `7` = "Mixed", `9` = "")) %>%
  mutate(hispanic = car::recode(as.numeric(VCF0104), "1:4 = 0; 5=1; 6:7 =0")) %>%
  mutate(idYear = c(1:nrow(.))) %>%
  mutate(stateFIPS = as.character(VCF0901a)) %>%
  mutate(stateFIPS = str_pad(stateFIPS, width = 2, side = "left", pad = "0")) %>%
  mutate(year = VCF0004) %>%
  mutate(org = "ANES") %>%
  select(pid3, ideology, age, sex = gender, race, hispanic, year, idYear, stateFIPS, org) %>%
  mutate(sex = as.character(sex)) %>%
  mutate(sex = ifelse(sex == "Male" | sex == "Female", sex, "")) %>%
  filter(year >= 2000)



In [38]:
fullData <- rbind(
  ANES %>% select(order(colnames(ANES))),
  CCESData %>% select(order(colnames(CCESData))),
  dat00 %>% select(order(colnames(dat00))),
  dat04 %>% select(order(colnames(dat04))),
  NAES08 %>% select(order(colnames(NAES08)))
) %>% filter(age <= 100) %>%
mutate(stateFIPS = ifelse(stateFIPS == "DC", "11", stateFIPS))


In [40]:
# Save the data
fullData %>% write.csv("pooledData/pooled.csv", row.names = FALSE)

# Estimate Some Stuff


In [15]:
library(dplyr)
dat = read.csv("/Users/Chris/Dropbox/masterData/pooledData/pooled.csv")

dat <- dat %>%
  mutate(ideo3 = ifelse(ideology == "", NA, ideology)) %>%
  mutate(pid3 = ifelse(pid3 == "", NA, pid3)) %>%
  select(ideo3, pid3, year, stateFIPS, org) %>%
  filter(year == 2000 | year == 2004 | year == 2008 | year == 2012 | year == 2016 | year == 2020) %>%
  filter(stateFIPS != "DC") %>%
  filter(stateFIPS!= "0")
dim(dat)

[1] 428195      5

In [18]:
library(brms)
library(tidybayes)
model1 <- brm(pid3 ~ 1 +
  (1| year) + (1 | stateFIPS),
data = dat,
family = categorical(link = "logit"),
algorithm = "meanfield"
)

Start sampling



Chain 1: ------------------------------------------------------------
Chain 1: EXPERIMENTAL ALGORITHM:
Chain 1:   This procedure has not been thoroughly tested and may be unstable
Chain 1:   or buggy. The interface is subject to change.
Chain 1: ------------------------------------------------------------
Chain 1: 
Chain 1: 
Chain 1: 
Chain 1: Gradient evaluation took 0.454138 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 4541.38 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Begin eta adaptation.
Chain 1: Iteration:   1 / 250 [  0%]  (Adaptation)
Chain 1: Iteration:  50 / 250 [ 20%]  (Adaptation)
Chain 1: Iteration: 100 / 250 [ 40%]  (Adaptation)
Chain 1: Iteration: 150 / 250 [ 60%]  (Adaptation)
Chain 1: Iteration: 200 / 250 [ 80%]  (Adaptation)
Chain 1: Success! Found best value [eta = 1] earlier than expected.
Chain 1: 
Chain 1: Begin stochastic gradient ascent.
Chain 1:   iter             ELBO   delta_ELBO

In [27]:
library(dplyr)
model1$data  %>%
  add_linpred_draws(model1, draws = 1000) %>% head()
# expanded_dat_1 <- fixed_data %>%
#   group_by(year) %>%
#   mutate(authoritarianism = quantile(authoritarianism, 0.975)) %>%
#   add_linpred_draws(fit0b, draws = 1000) %>%
#   mutate(high_auth = .linpred) %>%
#   select(high_auth)

# expanded_dat_0$high_auth <- expanded_dat_1$high_auth
# expanded_dat_0$marginal <- plogis(expanded_dat_0$high_auth) - plogis(expanded_dat_0$low_auth)

# marginals <- expanded_dat_0 %>%
#   group_by(year) %>%
#   mutate(min = quantile(marginal, 0.025)) %>%
#   mutate(med = quantile(marginal, 0.50)) %>%
#   mutate(max = quantile(marginal, 0.975)) %>%
#   summarize(
#     min = quantile(min, 0.025),
#     med = quantile(med, 0.50),
#     max = quantile(max, 0.975)
#   )
# marginals

In [23]:
library(brms)
library(tidybayes)

model1 <- brm(pid3 ~ ideo3 +
  (1  + ideo3| year) + (1 + ideo3 | stateFIPS),
data = dat,
family = categorical(link = "logit"),
seed = 1234,
iter = 5000
)

model2 <- brm(pid3 ~ ideo3 + as.factor(year) +
 (1 + ideo3 | stateFIPS),
data = dat,
family = categorical(link = "logit"),
seed = 1234,
iter = 5000
)

Warning message:
"Rows containing NAs were excluded from the model."
Compiling Stan program...

Start sampling



In [46]:
model <- brm(ideo3 ~ pid3 +
  (1 | year) + (1 | stateFIPS) + (1 | org),
  data = filterDat,
  family = categorical(link = "logit"),
  algorithm = "vb",
  seed = 1234,
  iter = 1000
)


# Sandbox


In [116]:
library(Rcpp)
#>
#> Attaching package: 'Rcpp'
#> The following object is masked from 'package:inline':
#>
#>     registerPlugin
cppFunction("int add(int x, int y, int z) {
  int sum = x + y + z;
  return sum;
}")
# add works like a regular R function
add
#> function (x, y, z)
#> .Call(<pointer: 0x7f96ecb3ef20>, x, y, z)
add(1, 2, 3)
#> [1] 6


function (x, y, z) 
.Call(<pointer: 0x10fee7c10>, x, y, z)

[1] 6

In [123]:
signR <- function(x) {
  if (x > 0) {
    1
  } else if (x == 0) {
    0
  } else {
    -1
  }
}
# These two functions are equivalent. But the C++ version is much faster.
cppFunction("int signC(int x) {
  if (x > 0) {
    return 1;
  } else if (x == 0) {
    return 0;
  } else {
    return -1;
  }
}")

x <- 2
signC(x)


[1] 1

In [ ]:
# Vectors
pdistR <- function(x, ys) {
  sqrt((x - ys)^2)
}

cppFunction("NumericVector pdistC(double x, NumericVector ys) {
  int n = ys.size();
  NumericVector out(n);

  for(int i = 0; i < n; ++i) {
    out[i] = sqrt(pow(ys[i] - x, 2.0));
  }
  return out;
}")
# Also useful
# NumericVector zs = clone(ys).


In [125]:
cppFunction("NumericVector rowSumsC(NumericMatrix x) {
  int nrow = x.nrow(), ncol = x.ncol();
  NumericVector out(nrow);

  for (int i = 0; i < nrow; i++) {
    double total = 0;
    for (int j = 0; j < ncol; j++) {
      total += x(i, j);
    }
    out[i] = total;
  }
  return out;
}")
# In C++, you subset a matrix with (), not [].
set.seed(1014)
x <- matrix(sample(100), 10)
rowSums(x)
#>  [1] 446 514 480 514 352 627 525 586 572 434
rowSumsC(x)


[1] 446 514 480 514 352 627 525 586 572 434

[1] 446 514 480 514 352 627 525 586 572 434

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
List irt2PL(NumericMatrix X, NumericVector Y, int nIter = 1000, double tol = 1e-6) {
  int n = X.nrow(), p = X.ncol();
  NumericVector beta(p), theta(n), eta(n), prob(n), loglik(nIter);
  double a = 1, b = 0.1, ll = 0, llOld = -INFINITY;

  for (int iter = 0; iter < nIter; iter++) {
    // E-step
    for (int i = 0; i < n; i++) {
      eta[i] = X(i, _) * beta + theta[i];
      prob[i] = exp(a * eta[i] - b * pow(eta[i], 2)) / (1 + exp(a * eta[i] - b * pow(eta[i], 2)));
    }

    // M-step
    for (int j = 0; j < p; j++) {
      double num = 0, denom = 0;
      for (int i = 0; i < n; i++) {
        num += X(i, j) * (Y[i] - prob[i]);
        denom += pow(X(i, j), 2) * prob[i] * (1 - prob[i]);
      }
      beta[j] += num / denom;
    }

    for (int i = 0; i < n; i++) {
      double num = 0, denom = 0;
      for (int j = 0; j < p; j++) {
        num += X(i, j) * beta[j];
        denom += pow(X(i, j), 2) * a * exp(a * eta[i] - b * pow(eta[i], 2));
      }
      theta[i] += (Y[i] - exp(num)) / denom;
    }

    // Compute log-likelihood
    ll = 0;
    for (int i = 0; i < n; i++) {
      ll += Y[i] * eta[i] - log(1 + exp(eta[i]));
    }
    loglik[iter] = ll;

    // Check convergence
    if (ll - llOld < tol) {
      break;
    }
    llOld = ll;
  }

  return List::create(Named("beta") = beta,
                      Named("theta") = theta,
                      Named("loglik") = loglik);
}


In [133]:
install.packages("maxLik")


In [130]:
library(maxLik)

# Define the log-likelihood function for the two parameter logit model
loglik <- function(beta, x, y) {
  eta <- beta[1] + beta[2] * x
  p <- exp(eta) / (1 + exp(eta))
  sum(y * log(p) + (1 - y) * log(1 - p))
}

# Define the EM algorithm
em_logit <- function(x, y, beta_init, n_iter = 1000, tol = 1e-6) {
  beta <- beta_init
  loglik_old <- -Inf
  for (i in 1:n_iter) {
    # E-step
    eta <- beta[1] + beta[2] * x
    p <- exp(eta) / (1 + exp(eta))

    # M-step
    beta_new <- mle2(loglik, start = list(beta = beta), method = "BFGS", x = x, y = y)$coef

    # Check convergence
    loglik_new <- loglik(beta_new, x, y)
    if (loglik_new - loglik_old < tol) {
      break
    }
    beta <- beta_new
    loglik_old <- loglik_new
  }
  return(beta)
}

# Test the EM algorithm on simulated data
set.seed(123)
n <- 10
x <- rnorm(n)
beta_true <- c(-1, 2)
p <- exp(beta_true[1] + beta_true[2] * x) / (1 + exp(beta_true[1] + beta_true[2] * x))
y <- rbinom(n, 1, p)
beta_init <- c(0, 0)
beta_est <- em_logit(x, y, beta_init)
beta_est


ERROR: Error in library(bbmle): there is no package called 'bbmle'


In [ ]:
# Define the log-likelihood function for the two parameter logit model
loglik <- function(beta, x, y) {
  eta <- beta[1] + beta[2] * x
  p <- exp(eta) / (1 + exp(eta))
  sum(y * log(p) + (1 - y) * log(1 - p))
}

# Define the EM algorithm
em_logit <- function(x, y, beta_init, n_iter = 1000, tol = 1e-6) {
  beta <- beta_init
  loglik_old <- -Inf
  for (i in 1:n_iter) {
    # E-step
    eta <- beta[1] + beta[2] * x
    p <- exp(eta) / (1 + exp(eta))

    # M-step
    beta_new <- mle2(loglik, start = list(beta = beta), method = "BFGS", x = x, y = y)$coef

    # Check convergence
    loglik_new <- loglik(beta_new, x, y)
    if (loglik_new - loglik_old < tol) {
      break
    }
    beta <- beta_new
    loglik_old <- loglik_new
  }
  return(beta)
}

# Test the EM algorithm on simulated data
set.seed(123)
n <- 100
x <- rnorm(n)
beta_true <- c(-1, 2)
p <- exp(beta_true[1] + beta_true[2] * x) / (1 + exp(beta_true[1] + beta_true[2] * x))
y <- rbinom(n, 1, p)
beta_init <- c(0, 0)
beta_est <- em_logit(x, y, beta_init)
beta_est


In [4]:
# Load Rcpp
library(Rcpp)

# Evaluate 2 + 2 in C++
x <- evalCpp("2 + 2")

# Evaluate 2 + 2 in R
y <- 2 + 2

# Storage modes of x and y
storage.mode(x)
storage.mode(y)

# Change the C++ expression so that it returns a double
z <- evalCpp("2.0 + 2.0")


[1] "integer"

[1] "double"

In [5]:
# Define the function euclidean_distance()
library(Rcpp)

cppFunction("
  double euclidean_distance(double x, double y) {
    return sqrt(x*x + y*y) ;
  }
")

# Calculate the euclidean distance
euclidean_distance(1.5, 2.5)


[1] 2.915476

In [7]:
library(Rcpp)

# Define the function add()
cppFunction('
  int add(int x, int y) {
    int res = x + y ;
    Rprintf("** %d + %d = %d\\n", x, y, res) ;
    return res ;
  }
')

# Call add() to print THE answer
add(40, 2)

cppFunction('
  // adds x and y, but only if they are positive
  int add_positive_numbers(int x, int y) {
      // if x is negative, stop
      if(x < 0) stop("x is negative") ;

      // if y is negative, stop
      if(y < 0) stop("y is negative") ;

      return x + y ;
  }
')

#include <Rcpp.h>
using namespace Rcpp;
#don't export everything to R.
// Make square() accept and return a double
double square(double x) {
  // Return x times x
  return x * x;
}

// [[Rcpp::export]]
double dist(double x, double y) {
  // Change this to use square()
  return sqrt(square(x) + square(y));
}

#include <Rcpp.h>
using namespace Rcpp;

double square(double x) {
  return x * x ;
}

// [[Rcpp::export]]
double dist(double x, double y) {
  return sqrt(square(x) + square(y));
}

// Start the Rcpp R comment block
/***
# Call dist() to the point (3, 4)
dist(3,4)
# Close the Rcpp R comment block
*/


** 40 + 2 = 42


[1] 42

In [ ]:
#include <Rcpp.h>
using namespace Rcpp ;

// [[Rcpp::export]]
double absolute(double x) {
  // Test for x greater than zero
  if(x > 0) {
    // Return x
    return x;
  // Otherwise
  } else {
    // Return negative x
    return -x;
  }
}

/*** R
absolute(pi)
absolute(-3)
*/

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
List create_vectors() {
  // Create an unnamed character vector
  CharacterVector polygons = CharacterVector::create("triangle", "square", "pentagon");
  // Create a named integer vector
  IntegerVector mersenne_primes = IntegerVector::create(_["first"] = 3, _["second"] = 7, _["third"] = 31);
  // Create a named list
  List both = List::create(_["polygons"] = polygons, _["mersenne_primes"] = mersenne_primes);
  return both;
}

/*** R
create_vectors()
*/

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
List change_negatives_to_zero(NumericVector the_original) {
  // Set the copy to the original
  NumericVector the_copy = the_original;
  int n = the_original.size();
  for(int i = 0; i < n; i++) {
    if(the_copy[i] < 0) the_copy[i] = 0;
  }
  return List::create(_["the_original"] = the_original, _["the_copy"] = the_copy);
}

// [[Rcpp::export]]
List change_negatives_to_zero_with_cloning(NumericVector the_original) {
  // Clone the original to make the copy
  NumericVector the_copy = clone(the_original);
  int n = the_original.size();
  for(int i = 0; i < n; i++) {
    if(the_copy[i] < 0) the_copy[i] = 0;
  }
  return List::create(_["the_original"] = the_original, _["the_copy"] = the_copy);
}

/*** R
x <- c(0, -4, 1, -2, 2, 4, -3, -1, 3)
change_negatives_to_zero(x)
# Need to define x again because it's changed now
x <- c(0, -4, 1, -2, 2, 4, -3, -1, 3)
change_negatives_to_zero_with_cloning(x)
*/

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
double weighted_mean_cpp(NumericVector x, NumericVector w) {
  // Initialize these to zero
  double total_w = 0;
  double total_xw = 0;

  // Set n to the size of x
  int n = x.size();

  // Specify the for loop arguments
  for(int i = 0; i<n; i++) {
    // Add ith weight
    total_w += w[i];
    // Add the ith data value times the ith weight
    total_xw += x[i]*w[i];
  }

  // Return the total product divided by the total weight
  return total_xw/total_w;
}

/*** R
x <- c(0, 1, 3, 6, 2, 7, 13, 20, 12, 21, 11)
w <- 1 / seq_along(x)
weighted_mean_cpp(x, w)
# Does the function give the same results as R's weighted.mean() function?
all.equal(weighted_mean_cpp(x, w), weighted.mean(x, w))
*/

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
double weighted_mean_cpp(NumericVector x, NumericVector w) {
  double total_w = 0;
  double total_xw = 0;

  int n = x.size();

  for(int i = 0; i < n; i++) {
    // If the ith element of x or w is NA then return NA
    if(NumericVector::is_na(x[i]) || NumericVector::is_na(w[i])) {
      return NumericVector::get_na();
    }
    total_w += w[i];
    total_xw += x[i] * w[i];
  }

  return total_xw / total_w;
}

/*** R
x <- c(0, 1, 3, 6, 2, 7, 13, NA, 12, 21, 11)
w <- 1 / seq_along(x)
weighted_mean_cpp(x, w)
*/

In [ ]:
# This is terrible code, because it's like appending alist. But C is cheaper by not making copies of the vector with each append.

#include <Rcpp.h>
using namespace Rcpp;

// Set the return type to a standard double vector
// [[Rcpp::export]]
std::vector<double> select_positive_values_std(NumericVector x) {
  int n = x.size();

  // Create positive_x, a standard double vector
  std::vector<double> positive_x;

  for(int i = 0; i < n; i++) {
    if(x[i] > 0) {
      // Append the ith element of x to positive_x
      positive_x.push_back(x[i]);
    }
  }
  return positive_x;
}

/*** R
set.seed(42)
x <- rnorm(1e6)
# Does it give the same answer as R?
all.equal(select_positive_values_std(x), x[x > 0])
# Which is faster?
microbenchmark(
  good_cpp = good_select_positive_values_cpp(x),
  std = select_positive_values_std(x)
)
*/

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
NumericVector positive_rnorm(int n, double mean, double sd) {
  // Specify out as a numeric vector of size n
  NumericVector out(n);

  // This loops over the elements of out
  for(int i = 0; i < n; i++) {
    // This loop keeps trying to generate a value
    do {
      // Call R's rnorm()
      out[i] = R::rnorm(mean, sd);
      // While the number is negative, keep trying
    } while(out[i]<=0);
  }
  return out;
}

/*** R
  positive_rnorm(10, 2, 2)
*/

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// From previous exercise; do not modify
// [[Rcpp::export]]
int choose_component(NumericVector weights, double total_weight) {
  double x = R::runif(0, total_weight);
  int j = 0;
  while(x >= weights[j]) {
    x -= weights[j];
    j++;
  }
  return j;
}

// [[Rcpp::export]]
NumericVector rmix(int n, NumericVector weights, NumericVector means, NumericVector sds) {
  // Check that weights and means have the same size
  int d = weights.size();
  if(means.size() != d) {
    stop("means size != weights size");
  }
  // Do the same for the weights and std devs
  if(sds.size() != d) {
    stop("sds size != weights size");
  }

  // Calculate the total weight
  double total_weight = sum(weights);

  // Create the output vector
  NumericVector res(n);

  // Fill the vector
  for(int i = 0; i < n; i++) {
    // Choose a component
    int j = choose_component(weights, total_weight);

    // Simulate from the chosen component
    res[i] = R::rnorm(means[j], sds[j]);
  }

  return res;
}

/*** R
  weights <- c(0.3, 0.7)
  means <- c(2, 4)
  sds <- c(2, 4)
  rmix(10, weights, means, sds)
*/

In [3]:
x <- rnorm(5, 0, 1)
rollmean1 <- function(x, window = 3) {
  n <- length(x)
  res <- rep(NA, n)
  for (i in seq(window, n)) {
    res[i] <- mean(x[seq(i - window + 1, i)])
    print(res[i])
  }
  res
}

rollmean1(x, 3)


[1] 0.2493567
[1] -0.08088453
[1] 0.2008669


[1]          NA          NA  0.24935667 -0.08088453  0.20086694

In [ ]:
# From previous step; do not modify
rollmean3 <- function(x, window = 3) {
  initial_total <- sum(head(x, window))
  lasts <- tail(x, -window)
  firsts <- head(x, -window)
  other_totals <- initial_total + cumsum(lasts - firsts)
  c(rep(NA, window - 1), initial_total / window, other_totals / window)
}

# This checks rollmean1() and rollmean2() give the same result
all.equal(rollmean1(x), rollmean2(x))

# This checks rollmean1() and rollmean3() give the same result
all.equal(rollmean1(x), rollmean3(x))

# Benchmark the performance
microbenchmark(
  rollmean1(x),
  rollmean2(x),
  rollmean3(x),
  times = 5
)


In [ ]:
# A C++ Rolling Mean

#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
NumericVector rollmean4(NumericVector x, int window) {
  int n = x.size();

  // Set res as a NumericVector of NAs with length n
  NumericVector res(n, NumericVector::get_na());

  // Sum the first window worth of values of x
  double total = 0.0;
  for(int i = 0; i < window; i++) {
    total += x[i];
  }

  // Treat the first case seperately
  res[window - 1] = total / window;

  // Iteratively update the total and recalculate the mean
  for(int i = window; i < n; i++) {
    // Remove the (i - window)th case, and add the ith case
    total += - x[i - window] + x[i];
    // Calculate the mean at the ith position
    res[i] = total / window;
  }

  return res;
}

/*** R
   # Compare rollmean2, rollmean3 and rollmean4
   set.seed(42)
   x <- rnorm(10000)
   microbenchmark(
    rollmean2(x, 4),
    rollmean3(x, 4),
    rollmean4(x, 4),
    times = 5
   )
*/

The function na_locf2 takes a NumericVector x as an argument. The NumericVector is a data type in Rcpp that corresponds to the numeric vector in R.

The variable current is initialized to NA using NumericVector::get_na(). This is the equivalent of current <- NA in R.

The size of the vector x is stored in the variable n. A new NumericVector res is created as a clone of x. This is similar to res <- x in R.

Then, a for loop is used to iterate over each element in x. If the ith element of x is NA (checked using NumericVector::is_na(x[i])), the ith element of res is set to current. This is the equivalent of res[i] <- current in R when is.na(x[i]) is TRUE.

If the ith element of x is not NA, current is set to the ith element of x. This is the equivalent of current <- x[i] in R when is.na(x[i]) is FALSE.

Finally, the function returns res, which is a vector that replaces all NA values in x with the last observed non-NA value.

The R code at the end is used to benchmark the performance of the Rcpp function na_locf2 against the R function na_locf1. It generates a random numeric vector x with some NA values, and then measures the time taken to execute each function a few times. This is not part of the solution to the exercise, but it's a good practice to check the performance of your code.


In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
NumericVector na_locf2(NumericVector x) {
  // Initialize to NA
  double current = NumericVector::get_na();

  int n = x.size();
  NumericVector res = clone(x);
  for(int i = 0; i < n; i++) {
    // If ith value of x is NA
    if(NumericVector::is_na(x[i])) {
      // Set ith result as current
      res[i] = current;
    } else {
      // Set current as ith value of x
      current = x[i];
    }
  }
  return res ;
}

/*** R
  library(microbenchmark)
  set.seed(42)
  x <- rnorm(1e5)
  # Sprinkle some NA into x
  x[sample(1e5, 100)] <- NA
  microbenchmark(
    na_locf1(x),
    na_locf2(x),
    times = 5
  )
*/

"The R version needs to create an intermediate vector the same length as y (x - ys), and allocating memory is an expensive operation. The C++ function avoids this overhead because it uses an intermediate scalar."


In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
NumericVector na_meancf2(NumericVector x) {
  double total_not_na = 0.0;
  double n_not_na = 0.0;
  NumericVector res = clone(x);

  int n = x.size();
  for(int i = 0; i < n; i++) {
    // If ith value of x is NA
    if(NumericVector::is_na(x[i])) {
      // Set the ith result to the total of non-missing values
      // divided by the number of non-missing values
      res[i] = total_not_na / n_not_na;
    } else {
      // Add the ith value of x to the total of non-missing values
      total_not_na += x[i];
      // Add 1 to the number of missing values
      n_not_na++;
    }
  }
  return res;
}

/*** R
  library(microbenchmark)
  set.seed(42)
  x <- rnorm(1e5)
  x[sample(1e5, 100)] <- NA
  microbenchmark(
    na_meancf1(x),
    na_meancf2(x),
    times = 5
  )
*/

In [ ]:
# An AR model
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
NumericVector ar2(int n, double c, NumericVector phi, double eps) {
  int p = phi.size();
  NumericVector x(n);

  // Loop from p to n
  for(int i = p; i < n; i++) {
    // Generate a random number from the normal distribution
    double value = R::rnorm(c, eps);
    // Loop from zero to p
    for(int j = 0; j < p; j++) {
      // Increase by the jth element of phi times
      // the "i minus j minus 1"th element of x
      value += phi[j] * x[i - j - 1];
    }
    x[i] = value;
  }
  return x;
}

/*** R
d <- data.frame(
  x = 1:50,
  y = ar2(50, 10, c(1, -0.5), 1)
)
ggplot(d, aes(x, y)) + geom_line()
*/

In [ ]:
ma1 <- function(n, mu, theta, sd) {
  q <- length(theta)
  x <- numeric(n)
  eps <- rnorm(n, 0, sd)
  for (i in seq(q + 1, n)) {
    value <- mu + eps[i]
    for (j in seq_len(q)) {
      value <- value + theta[j] * eps[i - j]
    }
    x[i] <- value
  }
  x
}


In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
NumericVector ma2(int n, double mu, NumericVector theta, double sd) {
  int q = theta.size();
  NumericVector x(n);

  // Generate the noise vector
  NumericVector eps = rnorm(n, 0.0, sd);

  // Loop from q to n
  for(int i = q; i < n; i++) {
    // Value is mean plus noise
    double value = mu + eps[i];
    // Loop from zero to q
    for(int j = 0; j < q; j++) {
      // Increase by the jth element of theta times
      // the "i minus j minus 1"th element of eps
      value += theta[j] * eps[i - j - 1];
    }
    // Set ith element of x to value
    x[i] = value;
  }
  return x;
}

/*** R
d <- data.frame(
  x = 1:50,
  y = ma2(50, 10, c(1, -0.5), 1)
)
ggplot(d, aes(x, y)) + geom_line()
*/

In [ ]:
This solution solves the exercise by implementing the instructions step by step in Rcpp.

First, it defines a function ma2 that takes four arguments: n (the size of the vector), mu (the mean), theta (a vector of coefficients), and sd (the standard deviation).

The function starts by initializing a vector x of size n and a vector eps of size n with random numbers generated from a normal distribution with mean 0 and standard deviation sd. This is done using the rnorm() function from the Rcpp namespace, as per the first instruction.

Then, it enters a loop that runs from q (the size of theta) to n. Inside this loop, it calculates value as mu plus the ith noise value, as per the second instruction.

Next, it enters a nested loop that runs from 0 to q. Inside this loop, it increases value by the jth element of theta times the "i minus j minus 1"th element of eps, as per the third instruction.

After the loops, it sets the ith element of x to value, as per the fourth instruction.

Finally, it returns the vector x.

The R code at the end of the script is used to test the function by generating a data frame d with two columns: x (a sequence from 1 to 50) and y (the result of the ma2 function). It then plots y against x using ggplot2.

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
NumericVector ma2(int n, double mu, NumericVector theta, double sd) {
  int q = theta.size();
  NumericVector x(n);

  // Generate the noise vector
  NumericVector eps = rnorm(n, 0.0, sd);

  // Loop from q to n
  for(int i = q; i < n; i++) {
    // Value is mean plus noise
    double value = mu + eps[i];
    // Loop from zero to q
    for(int j = 0; j < q; j++) {
      // Increase by the jth element of theta times
      // the "i minus j minus 1"th element of eps
      value += theta[j] * eps[i - j - 1];
    }
    // Set ith element of x to value
    x[i] = value;
  }
  return x;
}

/*** R
d <- data.frame(
  x = 1:50,
  y = ma2(50, 10, c(1, -0.5), 1)
)
ggplot(d, aes(x, y)) + geom_line()
*/

In [ ]:
#include <Rcpp.h>
using namespace Rcpp;

// [[Rcpp::export]]
NumericVector arma(int n, double mu, NumericVector phi, NumericVector theta, double sd) {
  int p = phi.size();
  int q = theta.size();
  NumericVector x(n);

  // Generate the noise vector
  NumericVector eps = rnorm(n, 0.0, sd);

  // Start at the max of p and q plus 1
  int start = std::max(p, q) + 1;

  // Loop i from start to n
  for(int i = start; i < n; i++) {
    // Value is mean plus noise
    double value = mu + eps[i];

    // The MA(q) part
    for(int j = 0; j < q; j++) {
      // Increase by the jth element of theta times
      // the "i minus j minus 1"th element of eps
      value += theta[j] * eps[i - j - 1];
    }

    // The AR(p) part
    for(int j = 0; j < p; j++) {
      // Increase by the jth element of phi times
      // the "i minus j minus 1"th element of x
      value += phi[j] * x[i - j - 1];
    }

    x[i] = value;
  }
  return x;
}

/*** R
d <- data.frame(
  x = 1:50,
  y = arma(50, 10, c(1, -0.5), c(1, -0.5), 1)
)
ggplot(d, aes(x, y)) + geom_line()
*/